In [310]:
import os
import cv2
import numpy as np
import pandas as pd

In [311]:
# Directories
predicted_masks_dir = "./grayscale_images_aug"
SIZE_X = 300
SIZE_Y = 300

In [312]:
def load_images_from_directory(directory):
    predicted_masks = []
    for filename in os.listdir(directory):
        full_path = os.path.join(directory, filename)  # Join directory and filename.
        mask = cv2.imread(full_path, 0)  # Read the image in grayscale.
        if mask is None:
            print(f"Failed to load image: {filename}")
            continue  # Skip this file
        RESIZE = cv2.resize(mask, (SIZE_X, SIZE_Y), interpolation=cv2.INTER_NEAREST)
        predicted_masks.append(RESIZE)
    return predicted_masks

In [313]:
# Loading predicted masks
predicted_masks = load_images_from_directory(predicted_masks_dir)
predicted_masks = np.array(predicted_masks)
print(predicted_masks.shape)

(291, 300, 300)


In [314]:
# Loading train.csv
df = pd.read_csv('vinbigdata-chest-xray-abnormalities-detection/train.csv')
print(df.head())

                           image_id          class_name  class_id rad_id  \
0  50a418190bc3fb1ef1633bf9678929b3          No finding        14    R11   
1  21a10246a5ec7af151081d0cd6d65dc9          No finding        14     R7   
2  9a5094b2563a1ef3ff50dc5c7ff71345        Cardiomegaly         3    R10   
3  051132a778e61a86eb147c7c6f564dfe  Aortic enlargement         0    R10   
4  063319de25ce7edb9b1c6b8881290140          No finding        14    R10   

    x_min   y_min   x_max   y_max  
0     NaN     NaN     NaN     NaN  
1     NaN     NaN     NaN     NaN  
2   691.0  1375.0  1653.0  1831.0  
3  1264.0   743.0  1611.0  1019.0  
4     NaN     NaN     NaN     NaN  


<h1>General functions and variables</h1>

In [315]:
def transform_classes(input_classes):
    # dictionary mapping each class ID to its position in the output list
    class_to_index = {
        21: 0,   # Pneumothorax
        39: 1,   # Atelectasis
        57: 2,   # Infiltration
        75: 3,   # Other lesion
        93: 4,   # Pleural effusion
        111: 5,  # Cardiomegaly
        129: 6,  # Pulmonary fibrosis
        147: 7,  # Calcification
        165: 8,  # Pleural thickening
        183: 9,  # Aortic enlargement
        201: 10, # Consolidation
        219: 11, # Lung Opacity
        237: 12, # ILD
        255: 13  # Nodule/Mass
    }
    
    # Initialized output list with zeros
    output = [0] * 14
    
    # Set positions to 1 for classes present in the input
    for class_id in input_classes:
        if class_id in class_to_index:
            output[class_to_index[class_id]] = 1
    
    return output

In [316]:
columns = [
    "Pneumothorax", "Atelectasis", "Infiltration", "Other lesion", 
    "Pleural effusion", "Cardiomegaly", "Pulmonary fibrosis", "Calcification",
    "Pleural thickening", "Aortic enlargement", "Consolidation", 
    "Lung Opacity", "ILD", "Nodule/Mass"
]

<h1>Predicted values</h1>

**Prediction variables**

In [317]:
prediction_path_folder = predicted_masks_dir

**Prediction code**

In [318]:
# function to get unique classes from predicted mask
def predicted_values_func(image_path):
    image = cv2.imread(image_path, 0)
    if image is None:
        print(f"Failed to load image: {filename}")
        return None  # Skip this file
    unique_values = np.unique(image)
    # Filter out zero
    non_zero_values = unique_values[unique_values != 0]
    return non_zero_values

In [319]:
# initializing an empty DataFrame to store all predictions
df_predictions = pd.DataFrame(columns=columns)

In [320]:
# for each picture in testing dataset
for filename in os.listdir(prediction_path_folder):
    file_id = filename.split('.')[0]
    #parts = filename.split('_')
    #if len(parts) > 1:  # Check if the split was successful
    #    file_id_pred = parts[1].split('.')[0]  # Get the second part and then split by dot to remove the extension
    #print(file_id_pred)
    print(file_id)
    full_path = os.path.join(prediction_path_folder, filename)
    # get predicted classes
    predicted_values_folder = predicted_values_func(full_path)
    if predicted_values_folder is None:
        print(f"Skipping {filename} due to an error.")
        continue
    print("Predicted classes:", predicted_values_folder)
    # transform predicted classes to [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] format
    pred_output_classes = transform_classes(predicted_values_folder)
    print("Transformed classes:",pred_output_classes)
    # creating dataframe
    df_pred = pd.DataFrame([pred_output_classes], columns=columns, index=[file_id])
    # appending the current DataFrame to the main DataFrame
    df_predictions = pd.concat([df_predictions, df_pred])

51a8d7259a0a6deac20b7c4979a7e847
Predicted classes: [ 57 111 183]
Transformed classes: [0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0]
354bad318bdc4946fbb261867157901f
Predicted classes: [111 183]
Transformed classes: [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0]
134cd4124bd1d75563cb47732742e331
Predicted classes: []
Transformed classes: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
b37cd57257591d19366e5a6f23ceb8f6
Predicted classes: [183]
Transformed classes: [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
f608bb02a0b91160b07fb867bbec84ad
Predicted classes: []
Transformed classes: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
628ba9788c00a8fa5fd77992fa9f63ed
Predicted classes: [111 183]
Transformed classes: [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0]
719962d2d347b4d5a6641074d75b78ad
Predicted classes: [111 183]
Transformed classes: [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0]
b311e9ad56a71aadfcd8be7009111352
Predicted classes: [111 183]
Transformed classes: [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0]
402e63

Predicted classes: [57 93]
Transformed classes: [0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
4fa30afdf5d4bbfcfd9071e2a56e7a4b
Predicted classes: [ 57  93 183]
Transformed classes: [0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0]
b4faf90679534ac80ab9113365203dc0
Predicted classes: []
Transformed classes: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
59a7447696c775e5330241645060ad1c
Predicted classes: [183]
Transformed classes: [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
80caa435b6ab5edaff4a0a758ffaec6e
Predicted classes: [57 75 93]
Transformed classes: [0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
3dead8495b3fdcdde5912953e520b303
Predicted classes: [ 93 111 183]
Transformed classes: [0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0]
50937a47ccac712f2dd7e750207c16ba
Predicted classes: []
Transformed classes: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
656744b811b1f0e3c65dd457b3cd8cd9
Predicted classes: [111 183]
Transformed classes: [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0]
070d087c1badb18ee110e19cbe158a40
Pre

Predicted classes: [ 93 183]
Transformed classes: [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0]
30303a4b9ba969bea4e676cbd4a2496c
Predicted classes: [111]
Transformed classes: [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
ec6ec12533b8495bb7344d8895dd4f05
Predicted classes: []
Transformed classes: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
21086ab046da90a4526e5604670a96ce
Predicted classes: [111 183]
Transformed classes: [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0]
9da65d59721d58bd125a4fc94b6ffec6
Predicted classes: [111 183]
Transformed classes: [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0]
eacc53e9f724bd51a38d76306e6db8d2
Predicted classes: [57 93]
Transformed classes: [0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
163898fbc57f00f58ad27e72031a541f
Predicted classes: []
Transformed classes: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
343e1bdfaa62eb9f0148e3cd32aab124
Predicted classes: [57]
Transformed classes: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
43a9edf6cea5fec0b63f8a6d9e2a50d7
Predicted classes

In [321]:
print("\nDataFrame for Predicted Classes:")
df_predictions


DataFrame for Predicted Classes:


,Pneumothorax,Atelectasis,Infiltration,Other lesion,Pleural effusion,Cardiomegaly,Pulmonary fibrosis,Calcification,Pleural thickening,Aortic enlargement,Consolidation,Lung Opacity,ILD,Nodule/Mass
51a8d7259a0a6deac20b7c4979a7e847,0,0,1,0,0,1,0,0,0,1,0,0,0,0
354bad318bdc4946fbb261867157901f,0,0,0,0,0,1,0,0,0,1,0,0,0,0
134cd4124bd1d75563cb47732742e331,0,0,0,0,0,0,0,0,0,0,0,0,0,0
b37cd57257591d19366e5a6f23ceb8f6,0,0,0,0,0,0,0,0,0,1,0,0,0,0
f608bb02a0b91160b07fb867bbec84ad,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163898fbc57f00f58ad27e72031a541f,0,0,0,0,0,0,0,0,0,0,0,0,0,0
343e1bdfaa62eb9f0148e3cd32aab124,0,0,1,0,0,0,0,0,0,0,0,0,0,0
43a9edf6cea5fec0b63f8a6d9e2a50d7,0,0,0,0,1,0,0,0,0,0,0,0,0,0
0a411e44ec67f3d5d80a5b3d5a3edca3,0,0,0,0,0,1,0,0,0,1,0,0,0,0


In [322]:
# savind dataframe with predictions to excel
df_predictions.to_excel('classification_on_segmentation/df_prediction.xlsx', index=True)

<h1>True values</h1>

**True values variables**

In [323]:
shades_of_grey = {
    "Nodule/Mass": 255,
    "ILD": 237,
    "Lung Opacity": 219,
    "Consolidation": 201,
    "Aortic enlargement": 183,
    "Pleural thickening": 165,
    "Calcification": 147,
    "Pulmonary fibrosis": 129,
    "Cardiomegaly": 111,
    "Pleural effusion": 93,
    "Other lesion": 75,
    "Infiltration": 57,
    "Atelectasis": 39,
    "Pneumothorax": 21,
    "No finding": 0,
}

**True values code**

In [324]:
def true_values_func(image_id):
    filtered_df = df[df['image_id'] == image_id]
    class_names = filtered_df['class_name'].unique()
    shade_values = list(map(shades_of_grey.get, class_names))
    shade_values.sort()
    return shade_values

In [325]:
# initializing an empty DataFrame to store all true values
df_true_values = pd.DataFrame(columns=columns)

In [326]:
# for each picture in testing dataset
for filename in os.listdir(prediction_path_folder):
    file_id_true = filename.split('.')[0]
    #parts = filename.split('_')
    #if len(parts) > 1:  # Check if the split was successful
    #    file_id_pred = parts[1].split('.')[0]  # Get the second part and then split by dot to remove the extension
    #print(file_id_pred)
    print(file_id_true)
    # get predicted classes
    true_values_folder = true_values_func(file_id_true)
    print("True classes for image_id", file_id_true, ":", true_values_folder)
    # transform predicted classes to [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] format
    true_output_classes = transform_classes(true_values_folder)
    print("Transformed classes:",true_output_classes)
    # creating dataframe
    df_true = pd.DataFrame([true_output_classes], columns=columns, index=[file_id_true])
    # appending the current DataFrame to the main DataFrame
    df_true_values = pd.concat([df_true_values, df_true])

51a8d7259a0a6deac20b7c4979a7e847
True classes for image_id 51a8d7259a0a6deac20b7c4979a7e847 : [75, 111, 147, 183]
Transformed classes: [0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0]
354bad318bdc4946fbb261867157901f
True classes for image_id 354bad318bdc4946fbb261867157901f : [111, 165, 183]
Transformed classes: [0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0]
134cd4124bd1d75563cb47732742e331
True classes for image_id 134cd4124bd1d75563cb47732742e331 : [75, 129, 165]
Transformed classes: [0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0]
b37cd57257591d19366e5a6f23ceb8f6
True classes for image_id b37cd57257591d19366e5a6f23ceb8f6 : [75, 147, 183, 219]
Transformed classes: [0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0]
f608bb02a0b91160b07fb867bbec84ad
True classes for image_id f608bb02a0b91160b07fb867bbec84ad : [75, 129, 147, 165, 183]
Transformed classes: [0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0]
628ba9788c00a8fa5fd77992fa9f63ed
True classes for image_id 628ba9788c00a8fa5fd77992fa9f63ed : [93, 111, 129, 165, 

True classes for image_id 67c83db34f74fb6203fc52f789ec3a31 : [57, 75, 93, 111, 165, 201, 219, 255]
Transformed classes: [0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1]
b57e21ea64ba9736e0711bdbbe53c221
True classes for image_id b57e21ea64ba9736e0711bdbbe53c221 : [93, 129, 165, 183]
Transformed classes: [0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0]
75ffb5f8839bef4d8719d46c29c46064
True classes for image_id 75ffb5f8839bef4d8719d46c29c46064 : [111]
Transformed classes: [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
88974ca0acec2690c1e0377c4523662e
True classes for image_id 88974ca0acec2690c1e0377c4523662e : [75, 111]
Transformed classes: [0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
0a425edf1164ad0a73e8b092c4cc8b3b
True classes for image_id 0a425edf1164ad0a73e8b092c4cc8b3b : [39, 129, 147, 165, 183, 255]
Transformed classes: [0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1]
24b3b89454ece6db0ba1a5cb83f3421a
True classes for image_id 24b3b89454ece6db0ba1a5cb83f3421a : [57, 75, 93, 165, 183, 219]
Transformed class

True classes for image_id 5ff1e6bd7cb14179a70db035d3aa6ba6 : [39, 75, 129, 165]
Transformed classes: [0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0]
1ec44d3bc379cf915de2c69bcb325cc2
True classes for image_id 1ec44d3bc379cf915de2c69bcb325cc2 : [93, 111, 165, 183]
Transformed classes: [0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0]
e674608b3b3609bdf9a5c8017ddcacba
True classes for image_id e674608b3b3609bdf9a5c8017ddcacba : [75, 111, 129, 183, 219]
Transformed classes: [0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0]
64e323f8638652f7bf813a933ebcbf07
True classes for image_id 64e323f8638652f7bf813a933ebcbf07 : [183]
Transformed classes: [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
8d6a0dcadf3c7322e146e729f4ac4ebf
True classes for image_id 8d6a0dcadf3c7322e146e729f4ac4ebf : [93, 129, 165, 255]
Transformed classes: [0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1]
42c049fd05428f7d606e9da4a95a8c3b
True classes for image_id 42c049fd05428f7d606e9da4a95a8c3b : [75, 93, 129, 165, 219]
Transformed classes: [0, 0, 0, 1, 1

True classes for image_id 6c6266968277d8d98717a106dfb716c3 : [39, 75, 93, 147, 165, 219]
Transformed classes: [0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0]
7a350dc1bcdf2fb34fcc57f05adf9da7
True classes for image_id 7a350dc1bcdf2fb34fcc57f05adf9da7 : [111, 147, 165, 183]
Transformed classes: [0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0]
f9004576db9e423ddce7bc64608e2aed
True classes for image_id f9004576db9e423ddce7bc64608e2aed : [39, 129, 165]
Transformed classes: [0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0]
30a08276cab4bd3ba231b2307341b987
True classes for image_id 30a08276cab4bd3ba231b2307341b987 : [57, 75, 129, 183]
Transformed classes: [0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0]
81e1bdd75db3ed8260cd2a8a206107a4
True classes for image_id 81e1bdd75db3ed8260cd2a8a206107a4 : [165, 219, 255]
Transformed classes: [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1]
80234017eb6caeada9e14d421eb0b8f8
True classes for image_id 80234017eb6caeada9e14d421eb0b8f8 : [57, 111, 219]
Transformed classes: [0, 0, 1, 0, 0

In [327]:
print("\nDataFrame for True Classes:")
df_true_values


DataFrame for True Classes:


,Pneumothorax,Atelectasis,Infiltration,Other lesion,Pleural effusion,Cardiomegaly,Pulmonary fibrosis,Calcification,Pleural thickening,Aortic enlargement,Consolidation,Lung Opacity,ILD,Nodule/Mass
51a8d7259a0a6deac20b7c4979a7e847,0,0,0,1,0,1,0,1,0,1,0,0,0,0
354bad318bdc4946fbb261867157901f,0,0,0,0,0,1,0,0,1,1,0,0,0,0
134cd4124bd1d75563cb47732742e331,0,0,0,1,0,0,1,0,1,0,0,0,0,0
b37cd57257591d19366e5a6f23ceb8f6,0,0,0,1,0,0,0,1,0,1,0,1,0,0
f608bb02a0b91160b07fb867bbec84ad,0,0,0,1,0,0,1,1,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163898fbc57f00f58ad27e72031a541f,0,0,0,1,0,0,1,0,0,0,0,0,0,1
343e1bdfaa62eb9f0148e3cd32aab124,0,0,0,0,0,0,1,0,1,1,1,1,0,1
43a9edf6cea5fec0b63f8a6d9e2a50d7,0,0,0,0,1,0,1,0,1,1,0,0,1,0
0a411e44ec67f3d5d80a5b3d5a3edca3,0,0,0,0,0,1,0,1,0,1,0,0,1,0


In [328]:
# savind dataframe with true values to excel
df_true_values.to_excel('classification_on_segmentation/df_true_values.xlsx', index=True)

<h1>Evaluation</h1>

**Creating comparison dataframe**

In [329]:
def compare_classes(row, col):
    true = row[col+'_true']
    pred = row[col+'_pred']
    if true == 1 and pred == 1:
        return "TP"
    elif true == 0 and pred == 0:
        return "TN"
    elif true == 1 and pred == 0:
        return "FN"
    elif true == 0 and pred == 1:
        return "FP"

# Create a combined DataFrame for processing
df_combined = pd.concat([df_true_values.add_suffix('_true'), df_predictions.add_suffix('_pred')], axis=1)

# Apply comparison function and create a result DataFrame
result = pd.DataFrame(index=df_true_values.index)
for col in df_true_values.columns:
    result[col] = df_combined.apply(lambda row: compare_classes(row, col), axis=1)

In [330]:
result

,Pneumothorax,Atelectasis,Infiltration,Other lesion,Pleural effusion,Cardiomegaly,Pulmonary fibrosis,Calcification,Pleural thickening,Aortic enlargement,Consolidation,Lung Opacity,ILD,Nodule/Mass
51a8d7259a0a6deac20b7c4979a7e847,TN,TN,FP,FN,TN,TP,TN,FN,TN,TP,TN,TN,TN,TN
354bad318bdc4946fbb261867157901f,TN,TN,TN,TN,TN,TP,TN,TN,FN,TP,TN,TN,TN,TN
134cd4124bd1d75563cb47732742e331,TN,TN,TN,FN,TN,TN,FN,TN,FN,TN,TN,TN,TN,TN
b37cd57257591d19366e5a6f23ceb8f6,TN,TN,TN,FN,TN,TN,TN,FN,TN,TP,TN,FN,TN,TN
f608bb02a0b91160b07fb867bbec84ad,TN,TN,TN,FN,TN,TN,FN,FN,FN,FN,TN,TN,TN,TN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163898fbc57f00f58ad27e72031a541f,TN,TN,TN,FN,TN,TN,FN,TN,TN,TN,TN,TN,TN,FN
343e1bdfaa62eb9f0148e3cd32aab124,TN,TN,FP,TN,TN,TN,FN,TN,FN,FN,FN,FN,TN,FN
43a9edf6cea5fec0b63f8a6d9e2a50d7,TN,TN,TN,TN,TP,TN,FN,TN,FN,FN,TN,TN,FN,TN
0a411e44ec67f3d5d80a5b3d5a3edca3,TN,TN,TN,TN,TN,TP,TN,FN,TN,TP,TN,TN,FN,TN


In [331]:
result.to_excel('classification_on_segmentation/result.xlsx', index=True)

**Calculating metrics per class**

In [332]:
# for overall metrics
def count_sum_of_state(state, dataframe):
    df_numerical = dataframe.applymap(lambda x: 1 if x == state else 0)
    total_of_state = df_numerical.sum().sum()
    return total_of_state

In [333]:
# for metrics per class (column in result dataframe)
def count_sum_of_state_in_column(state, dataframe, column_name):
    # Select the column and apply the condition directly to this column
    column_data = dataframe[column_name]
    df_numerical = column_data.apply(lambda x: 1 if x == state else 0)
    # Sum the values where the condition is True
    total_of_state = df_numerical.sum()
    return total_of_state

In [334]:
# initializing an empty dataframe to store metrics per class
df_per_class_metrics = pd.DataFrame(columns=['precision', 'recall', 'f1-score', 'support'], index=[columns])

In [335]:
for column in columns:
    # Calculate TP and FP for the current class
    TP = count_sum_of_state_in_column("TP", result, column)
    TN = count_sum_of_state_in_column("TN", result, column)
    FP = count_sum_of_state_in_column("FP", result, column)
    FN = count_sum_of_state_in_column("FN", result, column)

    print(f"Class {column} - TP: {TP}, TN: {TN}, FP: {FP}, FN: {FN}")

    # precision
    if TP + FP > 0:
        precision_per_class = TP / (TP + FP)
    else:
        precision_per_class = 0  # Avoid division by zero
        
    # recall
    if TP + FN > 0:
        recall_per_class = TP / (TP + FN)
    else:
        recall_per_class = 0
        
    # f1-score
    if precision_per_class + recall_per_class > 0:
        f1score_per_class = 2 * precision_per_class * recall_per_class/ (precision_per_class + recall_per_class)
    else:
        f1score_per_class = 0 

    # Write the calculated metrics into the DataFrame
    df_per_class_metrics.loc[column, 'precision'] = precision_per_class
    df_per_class_metrics.loc[column, 'recall'] = recall_per_class
    df_per_class_metrics.loc[column, 'f1-score'] = f1score_per_class
    df_per_class_metrics.loc[column, 'support'] = df_true_values[column].sum()  # Add the support

    
print("Metrics per class:")
df_per_class_metrics

Class Pneumothorax - TP: 8, TN: 278, FP: 3, FN: 2
Class Atelectasis - TP: 3, TN: 270, FP: 3, FN: 15
Class Infiltration - TP: 45, TN: 177, FP: 50, FN: 19
Class Other lesion - TP: 4, TN: 187, FP: 10, FN: 90
Class Pleural effusion - TP: 48, TN: 194, FP: 9, FN: 40
Class Cardiomegaly - TP: 128, TN: 121, FP: 30, FN: 12
Class Pulmonary fibrosis - TP: 13, TN: 163, FP: 0, FN: 115
Class Calcification - TP: 0, TN: 253, FP: 0, FN: 38
Class Pleural thickening - TP: 7, TN: 151, FP: 0, FN: 133
Class Aortic enlargement - TP: 153, TN: 76, FP: 25, FN: 37
Class Consolidation - TP: 0, TN: 257, FP: 2, FN: 32
Class Lung Opacity - TP: 0, TN: 177, FP: 0, FN: 114
Class ILD - TP: 4, TN: 262, FP: 1, FN: 24
Class Nodule/Mass - TP: 0, TN: 227, FP: 0, FN: 64
Metrics per class:


,precision,recall,f1-score,support
Pneumothorax,0.727273,0.8,0.761905,10
Atelectasis,0.5,0.166667,0.25,18
Infiltration,0.473684,0.703125,0.566038,64
Other lesion,0.285714,0.042553,0.074074,94
Pleural effusion,0.842105,0.545455,0.662069,88
Cardiomegaly,0.810127,0.914286,0.85906,140
Pulmonary fibrosis,1.0,0.101562,0.184397,128
Calcification,0,0.0,0,38
Pleural thickening,1.0,0.05,0.095238,140
Aortic enlargement,0.859551,0.805263,0.831522,190


**Calculating overall metrics**

In [336]:
print("TP:", count_sum_of_state("TP", result))
print("TN:", count_sum_of_state("TN", result))
print("FP:", count_sum_of_state("FP", result))
print("FN:", count_sum_of_state("FN", result))

TP: 413
TN: 2793
FP: 133
FN: 735


In [337]:
TP = count_sum_of_state("TP", result)
TN = count_sum_of_state("TN", result)
FP = count_sum_of_state("FP", result)
FN = count_sum_of_state("FN", result)

In [338]:
accuracy = (TP + TN) / (TP + FP + FN + TN)

In [339]:
precision = TP / (TP + FP)

In [340]:
recall = TP / (TP + FN)

In [341]:
f1score = (2 * precision * recall) / (precision + recall)

In [342]:
print("Overall Metrics:")
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1score:.2f}")

Overall Metrics:
Accuracy: 0.79
Precision: 0.76
Recall: 0.36
F1-Score: 0.49
